<!-- dom:TITLE: Ordinary Differential Equations (ODE) -->
# Ordinary Differential Equations (ODE)
**Prepared as part of MOD510 Computational Engineering and Modeling**

Date: **Apr 23, 2024**

# ODE Notebook
Learning objectives:
* being able to implement an ODE solver in python

* quantify numerical uncertainty

* test different methods and have basic understanding of the strength and weaknesses of each method

## Runge-Kutta Methods
<!-- FIGURE: [fig-ode/rk_fig, width=800] Illustration of the Euler algorithm, and a motivation for using the slope a distance from the $t_n$.<div id="fig:ode:rk"></div> -->

The 2. order Runge-Kutta method is accurate to $h^2$, with an error term of order $h^3$ 
**The 2. order Runge-Kutta:**

$$
k_1=hf(y_n,t_n)\nonumber
$$

$$
k_2=hf(y_n+\frac{1}{2}k_1,t_n+h/2)\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:rk4"></div>

$$
\begin{equation}  
y_{n+1}=y_n+k_2\label{eq:ode:rk4} \tag{1}
\end{equation}
$$

The Runge-Kutta fourth order method is one of he most used methods, it is accurate to order $h^4$, and has an error of order $h^5$. 
**The 4. order Runge-Kutta:**

$$
k_1=hf(y_n,t_n)\nonumber
$$

$$
k_2=hf(y_n+\frac{1}{2}k_1,t_n+h/2)\nonumber
$$

$$
k_3=hf(y_n+\frac{1}{2}k_2,t_n+h/2)\nonumber
$$

$$
k_4=hf(y_n+k_3,t_n+h)\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:rk5"></div>

$$
\begin{equation}  
y_{n+1}=y_n+\frac{1}{6}(k_1+2k_2+2k_3+k_4)\label{eq:ode:rk5} \tag{2}
\end{equation}
$$

# Exercise: Implement the Runge-Kutta method
In the following we are going to model a contaminated lake using a mixing tank. As an example we are going to use Norways largest lake, Mjosa, see [figure 1](#fig:eode:mjosa). 

<!-- dom:FIGURE: [fig-ode/mjosa.png, width=400 frac=1.0] The location of Mjosa and a mixing tank. The mixing tank assumes that at all times the concentrations inside the tank is uniform.  <div id="fig:eode:mjosa"></div> -->
<!-- begin figure -->
<div id="fig:eode:mjosa"></div>

<img src="fig-ode/mjosa.png" width=400><p style="font-size: 0.9em"><i>Figure 1: The location of Mjosa and a mixing tank. The mixing tank assumes that at all times the concentrations inside the tank is uniform.</i></p>
<!-- end figure -->

The volume of Mjosa is, $V=$56 km$^3$, and the discharge is $q=321$ m$^3$/s=27.734$\cdot10^6\text{m}^3/\text{day}$. We will assume that some contaminant are present *uniformly* in the lake, and fresh water is flowing into Mjosa. Applying mass balance to the system, and assuming that the flow pattern in Mjosa is such that the contaminant is distributed uniformly at all times, the following equation should hold

<!-- Equation labels as ordinary links -->
<div id="eq:eode:cstr"></div>

$$
\begin{equation}
V\frac{dC(t)}{dt} = q(t)\left[C_\text{in}(t) - C(t)\right].
\label{eq:eode:cstr} \tag{3}
\end{equation}
$$

Assume that the initial concentration of the contaminant is $C_0=1$, and assume that water flowing into contains no contaminant, we have the boundary conditions $C_\text{in}(t)=0$, $C(0)=1$. In this case equation ([3](#eq:eode:cstr)) is

<!-- Equation labels as ordinary links -->
<div id="eq:eode:cstr2"></div>

$$
\begin{equation}
\frac{dC(t)}{dt} = -\frac{1}{\tau}C(t),
\label{eq:eode:cstr2} \tag{4}
\end{equation}
$$

where $\tau\equiv V/q$. The analytical solution is simply

<!-- Equation labels as ordinary links -->
<div id="eq:eode:ana"></div>

$$
\begin{equation}
C(t)=e^{-t/\tau}.
\label{eq:eode:ana} \tag{5}
\end{equation}
$$

**Part 1.**
In the following you are going to first implement a *general* solver, then you are to test it on equation ([5](#eq:eode:ana)). The solution of ODE equations are based on solving a generic equation of the form

<!-- Equation labels as ordinary links -->
<div id="eq:eode:gen"></div>

$$
\begin{equation}
\frac{dy(t)}{dt}=f(y,t).
\label{eq:eode:gen} \tag{6}
\end{equation}
$$

Thus the solver should take in *as argument*, the right hand side, $f(y,t)$, starting values $y_0$, the start time $t_0$ and end time $t_f$.

Complete the code below

In [1]:
%matplotlib inline


import matplotlib.pyplot as plt
import numpy as np

#global parameters
c_in=0
c0=1
q=321*24*60*60*365 #m^3/year
V=56*1000**3    #m^3
tau=V/q        #day

def func(y,t):
    """
    the right hand side of ode
    """
    return ...

def rk4_step(func,y,t,dt):
    """
    t : time
    dt : step size (dt=h)
    func : the right hand side of the ode
    """
    ... 
    return

def rk2_step(func, y, t, dt):
     """
    t : time
    dt : step size (dt=h)
    func : the right hand side of the ode
    """ 
    return 

def ode_solv(func,y0,dt,t0,t_final):
    y=[];t=[]
    ti=t0
    y_old=y0
    while(ti <= t_final):
        t.append(ti); y.append(y_old)
        y_new = y_old+rk4_step(func,y_old,ti,dt) # or rk2_step    
        y_old = y_new
        ti   += dt
    return np.array(t),np.array(y)

**Part 2.**
1. How much time does it take for the contaminant to drop to 10$\%$ of its original value?

2. Is this model a good model for the cleaning of Mjosa?

3. Does the numerical error scales as expected

<!-- --- begin exercise --- -->

## Exercise 1: Adaptive step size - Runge-Kutta Method

In this exercise you are going to improve the algorithms above by choosing a step size that is not too large or too small. This will serve two purposes i) *greatly* enhance the efficiency of the code, and ii) ensure that we find the correct numerical solution that is *close enough* to the true solution. We are going to use the following result from the compendium [[hiorth]](#hiorth) (to get a good understanding it is advised to derive them)

<!-- Equation labels as ordinary links -->
<div id="_auto1"></div>

$$
\begin{equation}
|\epsilon|=\frac{|\Delta|}{2^p-1}=\frac{|y_1^*-y_1|}{2^p-1},
\label{_auto1} \tag{7}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto2"></div>

$$
\begin{equation}  
dt^\prime=\beta dt\left|\frac{\epsilon^\prime}{\epsilon}\right|^{\frac{1}{p+1}},
\label{_auto2} \tag{8}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto3"></div>

$$
\begin{equation}  
\hat{y_1}=y_1-\epsilon=\frac{2^p y_1-y_1^*}{2^{p}-1},
\label{_auto3} \tag{9}
\end{equation}
$$

where $\epsilon^\prime$ is the desired accuracy. $\beta$ is a safety factor $\beta\simeq0.8,0.9$, and you should always be careful that the step size do not become too large so that
the method breaks down. This can happens when $\epsilon$ is very low, which may happen if $y_1^*\simeq y_1$ and/or if $y_1^*\simeq y_1\simeq 0$.  

**Part 1.**

Use the equations above, and implement an adaptive step size algorithm for the 4. order Runge-Kutta methods. Use the Mjosa example above to test your code. It might be a good idea to use a safety limit on the step size `min(dt*(tol/toli)**(0.2),dt_max)`, where `dt_max` is the maximum step size you allow. The tolerance should be calculated as $\epsilon^\prime = atol +|y|rtol$, where 'atol' is the absolute tolerance and 'rtol' is the relative tolerance. A sensible choice would be to set 'atol=rtol' (e.g. = $10^{-5}$).

Below is some code to help you get started

In [2]:
def rk_adpative(func,y0,t0,tf,rel_tol=1e-5,abs_tol=1e-5,p=4):
    y=[]
    t=[]
    ti=t0
    y.append(y0)
    t.append(ti)
    dt=1e-2 # start with a small step 
    while(ti<=tf):
        y_old=y[-1]
        EPS=np.abs(y_old)*rel_tol+abs_tol
        eps=10*EPS
        while(eps>EPS): # continue while loop until correct dt
            DT=dt
            y_new =  ....  # one large step from t to t+dt
            y1    =  ....  # and two small steps - from t -> t+dt/2
            y2    =  ....  # and from t+dt/2 to t + dt
            eps   =  ....  # estimate numerical error
            dt    =  ....  # calculate new time step
	    
        y.append( ... )
        ti=ti+DT # important to add DT not dt
        t.append(ti)
    return np.array(t),np.array(y) # cast to numpy arrays

**Part 2.**
1. How many steps do you need to get a reasonable solution? 

2. Is the numerical error what you expect?

<!-- --- end exercise --- -->

<!-- --- begin exercise --- -->

## Exercise 2: Solving a set of ODE equations

What happens if we have more than one equation that needs to be solved? If we continue with our current example, we might be interested in what would happen 
if we had multiple tanks in series. This could be a very simple model to describe the cleaning  of drinking water infiltrated by salt water (a typical challenge in many countries) by injecting fresh water into it. Assume that the lake was connected to two nearby fresh water lakes, as illustrated in [figure 2](#fig:ode:cstr3). The weakest part of the model is the assumption about 
complete mixing, in a practical situation we could enforce complete mixing with the salty water in the first tank by injecting fresh water at multiple point in the 
lake. For the two next lakes, the degree of mixing is not obvious, but salt water is heavier than fresh water and therefore it would sink and mix with the fresh water. Thus
if the discharge rate was slow, one might imaging that a more or less complete mixing could occur. Our model then could answer questions like, how long time would it take before most
of the salt water is removed from the first lake, and how much time would it take before most of the salt water was cleared from the whole system? The answer to 
these questions would give practical input on how much and how fast one should inject the fresh water to clean up the system. If we had 
data from an actual system, we could compare our model predictions with data from the physical system, and investigate if our model description was correct. 

<!-- dom:FIGURE: [fig-ode/cstr3.png, width=800] A simple model for cleaning a salty lake that is connected to two lakes down stream. <div id="fig:ode:cstr3"></div> -->
<!-- begin figure -->
<div id="fig:ode:cstr3"></div>

<img src="fig-ode/cstr3.png" width=800><p style="font-size: 0.9em"><i>Figure 2: A simple model for cleaning a salty lake that is connected to two lakes down stream.</i></p>
<!-- end figure -->

For simplicity we will assume that all the lakes have the same volume, $V$. The governing equations follows
as before, by assuming mass balance:

$$
C_0(t+\Delta t)\cdot V - C_0(t)\cdot V = q(t)\cdot C_\text{in}(t)\cdot \Delta t - q(t)\cdot C_0(t)\cdot \Delta t,\nonumber
$$

$$
C_1(t+\Delta t)\cdot V - C_1(t)\cdot V = q(t)\cdot C_0(t)\cdot \Delta t - q(t)\cdot C_1(t)\cdot \Delta t,\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:cstr3a"></div>

$$
\begin{equation}  
C_2(t+\Delta t)\cdot V - C_2(t)\cdot V = q(t)\cdot C_1(t)\cdot \Delta t - q(t)\cdot C_2(t)\cdot \Delta t.\label{eq:ode:cstr3a} \tag{10}
\end{equation}
$$

Taking the limit $\Delta t\to 0$, we can write equation ([10](#eq:ode:cstr3a)) as:

<!-- Equation labels as ordinary links -->
<div id="eq:ode:cstr3b"></div>

$$
\begin{equation}
V\frac{dC_0(t)}{dt} = q(t)\left[C_\text{in}(t) - C_0(t)\right],\label{eq:ode:cstr3b} \tag{11}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:cstr3c"></div>

$$
\begin{equation}  
V\frac{dC_1(t)}{dt} = q(t)\left[C_0(t) - C_1(t)\right],\label{eq:ode:cstr3c} \tag{12}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:cstr3d"></div>

$$
\begin{equation}  
V\frac{dC_2(t)}{dt} = q(t)\left[C_1(t) - C_2(t)\right].\label{eq:ode:cstr3d} \tag{13}
\end{equation}
$$

Show that the analytical solution is:

<!-- Equation labels as ordinary links -->
<div id="_auto4"></div>

$$
\begin{equation}
C_0(t)=C_{0,0}e^{-t/\tau}{\nonumber}
\label{_auto4} \tag{14}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto5"></div>

$$
\begin{equation}  
C_1(t)=C_{0,0}\frac{t}{\tau}e^{-t/\tau}{\nonumber}
\label{_auto5} \tag{15}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:ode:cstr3j"></div>

$$
\begin{equation}  
C_2(t)=\frac{C_{0,0}t^2}{2\tau^2}e^{-t/\tau}.\label{eq:ode:cstr3j} \tag{16}
\end{equation}
$$

The numerical solution follows the exact same pattern as before if we introduce a vector notation.

$$
\frac{d}{dt}
\left(
\begin{array}{c} 
 C_0(t)\\ 
 C_1(t)\\ 
 C_2(t)
 \end{array}
 \right)
=\frac{1}{\tau}\left(
\begin{array}{c} 
 C_\text{in}(t) - C_0(t)\\ 
 C_0(t) - C_1(t)\\ 
 C_1(t) - C_2(t)
 \end{array}
 \right),\nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="eq:eode:vec"></div>

$$
\begin{equation}  
 \frac{d\mathbf{C}(t)}{dt}=\mathbf{f}(\mathbf{C},t).
\label{eq:eode:vec} \tag{17}
\end{equation}
$$

**Part 1.**
1. Extend the code in the previous exercises to be able to handle vector equations - note that if you have consistently used Numpy arrays you should actually be able to run your code without any modifications! (not the Richardson extrapolation algorithm)

2. Solve the set of equations in equation ([17](#eq:eode:vec)), and compare with the analytical solution

<!-- --- end exercise --- -->

# Exercise: Adaptive method for a general ODE

**Part 1.**
In this exercise we ask you to extend your implementation of the Richardson extrapolation to also be valid if the right hand side is a vector. We will not give the solution to this exercise, but rather tell you exactly how to do it and then you can try for yourself. The following recipe applies to the suggested solution (see separate pdf document)
1. There are only minor changes to the code, first we need to consider `EPS=np.abs(y_old)*rel_tol+abs_tol`. This expression is ambiguous, because `y_old` is a vector and `EPS` should be a single number. We suggest to simply replace the absolute value with the norm $\sqrt{y_0^2+y_1^2+\cdots}$, which can be achieved by `EPS=np.linalg.norm(y_old)*rel_tol+abs_tol`

2. The same also applies to the line `eps   = np.abs(y2-y_new)/(2**p-1)`, and this should be changed to `eps   = np.linalg.norm(y2-y_new)/(2**p-1)`

3. Make the suggested changes and test your code on equation ([17](#eq:eode:vec))

# Exercise: Second order equations
Test your solver on the following equation

<!-- Equation labels as ordinary links -->
<div id="eq:eode:ss"></div>

$$
\begin{equation}
xy^{\prime\prime}(x)+2^\prime(x)+x=1,
\label{eq:eode:ss} \tag{18}
\end{equation}
$$

where the initial conditions are $y(1)=2$, and $y^\prime(1)=1$. The analytical solution is

<!-- Equation labels as ordinary links -->
<div id="_auto6"></div>

$$
\begin{equation}
y(x)=\frac{5}{2}-\frac{5}{6x}+\frac{x}{2}-\frac{x^2}{6}.
\label{_auto6} \tag{19}
\end{equation}
$$

## References

1. <div id="hiorth"></div> **A. Hiorth**.  *Computational Engineering and Modeling*, https://github.com/ahiorth/CompEngineering, 2019.